In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

# Tiempo de mezcla (mixing time)

Previamente hemos visto como diseñar una cadena de Markov finita tal que converja a una distribución estacionaria de nuestro interés

Pero ¿Cuánto debemos esperar para que ocurra la convergencia? 

El tiempo de mezcla para una cadena de Markov irreducible y aperiodica se define como

$$
t_{mix}(\epsilon) = \min \left(n > 0: \|s(n) - \pi\|_{TV} < \epsilon \right)
$$

es decir el mínimo tiempo (número de pasos) tal que estemos a una distancia $\epsilon$ de la distribución estacionaria $\pi$

El operador $\|p - q\|_{TV} = \max_{x\in\mathcal{\Omega}} \|p(x) - q(x)\|$ se conoce como la distancia de variación total entre dos distribuciones

Se tienen algunas garantías con respecto a este tiempo, en particular la siguiente cota superior

$$
t_{mix}(\epsilon) < \log \left(\frac{1}{\epsilon \sqrt{\min_j \pi_j}} \right) \frac{1}{1-\lambda_*} 
$$

donde $\lambda_*$ es el segundo valor propio más grande de la matriz de transición $P$ de la cadena. 

La descomposición en valores propios de la matriz de transición de una cadena irreducible y de $\mathcal{S}$ estados se puede escribir como

$$
P^n = \sum_{i=1}^\mathcal{S} \alpha_i \lambda_i^n = \pi + \sum_{i=2}^\mathcal{S} \alpha_i \lambda_i^n
$$

Por propiedad de la cadena de Markov irreducible su valor propio más grande siempre es igual a uno y su vector propio asociado es la distribución estacionaria. 

Todos los demás valores propios se harán eventualmente cero cuando $n \to \infty$, siendo el segundo valor propio más grande y distinto de uno el que más se demore


# Autocorrelación en la cadena y número de muestras efectivo

¿Cómo podemos confirmar si nuestro algoritmo MCMC ha convergido? 

Por construcción, las muestras de nuestra traza son dependientes, pues $\theta_{t+1}$ se calcula a partir de $\theta_t$. 

Sin embargo, luego de un periódo de *burn-in*, las probabilidades de transición de la cadena debería converger a la distribución estacionaria y volverse independientes del tiempo

Es decir que podemos confirmar la convergencia estudiando la autocorrelación de la traza

$$
\rho(\tau) = \mathbb{E}\left[(\theta_t - \bar \theta)(\theta_{t+\tau}  - \bar \theta)\right]
$$

La autocorrelación nos indica que tanto las muestras de una serie de tiempo dependen de muestras pasadas. 


En este caso, al graficar $\rho$ en función de $\tau$ buscamos una autocorrelación que converja rapidamente y que luego fluctue en torno a cero

<img src="images/autocorr.png" width="700">

## Ejemplo

La lección pasada vimos como el valor de $\sigma_\epsilon$ repercutía fuertemente en la convergencia del algoritmo de Metropolis

Usemos la función `np.correlate` para estudiar la autocorrelación de las trazas para distintos valores de $\sigma_\epsilon$

In [ ]:
x = np.array([9.37, 10.18, 9.16, 11.60, 10.33])

prior = lambda theta : scipy.stats.norm(loc=5, scale=np.sqrt(10)).pdf(theta)
likelihood = lambda theta : np.prod([scipy.stats.norm(loc=theta, scale=1.).pdf(x_) for x_ in x])
r = lambda ts, tt : likelihood(ts)*prior(ts)/(likelihood(tt)*prior(tt)) 

def metropolis(mix_time=5000, sigma_eps=1.):
    thetas = np.zeros(shape=(mix_time, ))
    thetas[0] = np.random.randn()
    ar = 0.
    qs = scipy.stats.norm(loc=0, scale=sigma_eps).rvs(size=mix_time)
    us = scipy.stats.uniform.rvs(size=mix_time)
    for n in range(1, mix_time):
        theta_star = thetas[n-1] + qs[n]    
        if us[n] < np.amin([1, r(theta_star, thetas[n-1])]):
            thetas[n] = theta_star
            ar += 1.
        else:
            thetas[n] = thetas[n-1]
    return thetas, ar/mix_time

def autocorr(thetas):
    thetas_norm = (thetas-np.mean(thetas))/np.std(thetas)
    rho = np.correlate(thetas_norm, 
                       thetas_norm, mode='full')
    return rho[len(rho) // 2:]/len(thetas)

def neff(rho):
    T = np.where(rho < 0.)[0][0]
    return len(rho)/(1 + 2*np.sum(rho[:T]))
    

In [ ]:
%%time

np.random.seed(12345)

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
for sigma in [0.1, 1., 2., 10.]:
    thetas, ar = metropolis(mix_time=2000, sigma_eps=sigma)
    ax[0].plot(thetas)    
    ax[1].plot(autocorr(thetas), label=str(sigma))
    print(f"Número efectivo de muestras para sigma {sigma}: {neff(autocorr(thetas)):0.4f}")
    print(f"Fracción de aceptación: {ar:0.4f}")

ax[0].set_title('Traza')
ax[1].set_title('Autocorrelación');
ax[1].legend();

De la figura podemos ver que 

- Si el paso de las propuestas es muy corto, todos los pasos son aceptados, pero la correlación entre los mismos es alta por ser poco diversos
- Si el paso de las propuestas es muy largo, se proponen demasiadas propuestas malas que terminan siendo rechazadas. La fracción de aceptación disminuye y la autocorrelación aumenta

La fracción de aceptación es la cantidad de propuestas aceptadas dividido las propuestas totales. La sugerencia de los expertos es calibrar el algoritmo de Metropolis tal que alcance [una fracción de aceptación cercana a 23.4%](https://www.maths.lancs.ac.uk/~sherlocc/Publications/rwm.final.pdf) 

Una figura de mérito muy utilizada que se basa en la función de autocorrelación es el número de muestras efectivas

$$
n_{eff} = \frac{N}{1 + 2 \sum_{\tau=1}^T \rho(\tau)}
$$

donde $N$ es la cantidad de muestras de la cadena y $T$ es el instante en que la autocorrelación se vuelve por primera vez negativa

Idealmente quisieramos que $n_{eff} = N$, pero debido a que las muestras no son independientes en realidad tendremos $n_{eff} < N$

Podemos calibrar nuestro algoritmo MCMC tal que maximicemos $n_{eff}$

## Thinning (adelgazamiento)

Es una técnica para disminuir la autocorrelación de la traza. 

Consiste en submuestrear la traza, retendiendo sólo cada $t$ muestras, donde $t$ es el "intervalo de adelgazamiento". La idea es escoger este intervalo estudiando la función de autocorrelación

Debido a la gran cantidad de muestras que se podrían descartar es preferible ajustar adecuadamente el paso de las propuestas por sobre esta técnica

## Estadístico Gelman-Rubin

Supongamos que no entrenamos una sino $M$ cadenas

Otra forma de estudiar la convergencia en este caso es el estadístico Gelman Rubin o $\hat r$

Este estadístico compara la varianza (dispersión) dentro de la cadena con la varianza entre las $M$ cadenas. 

Si el valor de $\hat r$ es cercano a uno es indicativo de que la cadena ha convergido

# Material extra

- [Valores propios](https://www.cl.cam.ac.uk/teaching/1819/Probablty/materials/Lecture9_handout.pdf) y [cotas en los tiempos de mezcla](https://www.cl.cam.ac.uk/teaching/1920/Probablty/materials/Lecture10.pdf)
- [Parallel tempering](https://www.pas.rochester.edu/~sybenzvi/courses/phy403/2016s/p403_18_mcmc.pdf)
- [Artículo "Una introducción conceptual a MCMC"](https://arxiv.org/pdf/1909.12313.pdf)